# Compare the base model with the new one

In [ ]:
import os
import boto3
from ultralytics import YOLO

In [ ]:
BASE_MODEL_BUCKET = "model-registry"
BASE_MODEL_PATH = "models/default/best.pt"
NEW_MODEL_PATH = "models/new-model.pt"

MAP_THRESHOLD = float(os.getenv("map_threshold", 0.8))

AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_S3_BUCKET = os.getenv("AWS_S3_BUCKET")
AWS_S3_ENDPOINT = os.getenv("AWS_S3_ENDPOINT")

In [ ]:
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)
client = session.client("s3", endpoint_url=AWS_S3_ENDPOINT)

## Get the models

In [ ]:
client.download_file(BASE_MODEL_BUCKET, BASE_MODEL_PATH, "/tmp/base-model.pt")
client.download_file(AWS_S3_BUCKET, NEW_MODEL_PATH, "/tmp/new-model.pt")
client.download_file("trains", "validation-dataset.tar.gz", "/tmp/validation-dataset.tar.gz")

In [ ]:
! tar -xzf /tmp/validation-dataset.tar.gz

## Base Model Results

In [ ]:
model = YOLO("/tmp/base-model.pt")
results = model.val(data = "utils/data.yaml", split="test")

In [ ]:
res = model("utils/sample-inference-example.jpg")
res[0].show()

## New Model Results

In [ ]:
new_model = YOLO("/tmp/new-model.pt")
results = new_model.val(data = "utils/data.yaml", split="test")

In [ ]:
res = new_model("utils/sample-inference-example.jpg")
res[0].show()

## Make the Pipeline Fail Based on a Minimal mAP50 Threshold

In [ ]:
assert results.results_dict["metrics/mAP50(B)"] >= MAP_THRESHOLD